In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import comb
from scipy.stats import hypergeom
from matplotlib import rcParams
from collections import defaultdict
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']
rcParams['pdf.fonttype'] = 42
rcParams['axes.formatter.useoffset'] = False

In [2]:
def formatAxes(ax,text_size=10,xlim=None,xlabel=None,xticks=None,xticklabels=None,ylim=None,yticks=None,ylabel=None,yticklabels=None):
    """ to tweak the plotting of the axes as well as the fontsize """
    for loc,spine in list(ax.spines.items()):
        if loc == 'left': # settings for the y-axis
            if yticklabels and not yticks:
                sys.stderr.write("Need to provide a list wiht both y-labels and y-ticks!")
            if yticks:
                ax.yaxis.set_ticks_position('left')
                ax.yaxis.set_tick_params(direction='out')
                spine.set_visible(True)
                spine.set_position(("outward",1))
                spine.set_smart_bounds(True)
                ax.set_yticks(yticks)
                if ylim:
                    ax.set_ylim(ylim)
                if yticklabels:
                    ax.set_yticklabels(yticklabels,fontsize=text_size)
            else:
                spine.set_visible(False)
                ax.set_yticklabels([])
                ax.tick_params(axis='y',which='both',length=0)
                
            if ylabel:
                ax.set_ylabel(ylabel,fontsize=text_size,rotation=0,labelpad=160)
                ax.get_yaxis().set_label_coords(-0.1,0.5)
        elif loc == 'bottom': # settings for x-axis
            if xticks:
                spine.set_position('zero')
                spine.set_visible(False)
                spine.set_smart_bounds(False)
                ax.set_xticks(xticks)
                ax.tick_params(axis='x',which='both',length=0)
                if xlim:
                    ax.set_xlim(xlim)
                if xticklabels:
                    ax.set_xticklabels(xticklabels,fontsize=text_size)
            else:
                spine.set_visible(False)
                ax.set_xticklabels([])
                ax.tick_params(axis='x',which='both',length=0)
                
            if xlabel:
                ax.tick_params(axis='x',which='both',length=0)
                ax.set_xlabel(xlabel,fontsize=text_size+2)
                ax.xaxis.labelpad = 10
        else:
            spine.set_visible(False)
    ax.patch.set_visible(False)

### Loading the big dataframe:

In [3]:
alldata = pd.read_csv('../../../New_data_table.txt',\
                      sep="\t",\
                      header=0,\
                      index_col=0)

### Masking positions not considered by deltaSHAPE:

### ROC 

In [4]:

#SETTING NUCLEOTIDES TO ZERO IF FALLING IN REGIONS NOT USED BY DIFFBUM HMM
proteins = ["CELF1","FUS","HuR","PTBP1","RBFOX2","TARDBP"]


for index, row in alldata.iterrows():
    if index>=1 and index<=78 or index>=2451 and index<=2599 or index>=2500 and index<=4500 or index>=17801 and index<=17918:
        for protein in proteins:
            row[protein]=0




In [5]:
del protein

In [12]:
#FOR ROC we need to plot true positive rate and false positive rate

#QUESTION: should I use SCALED or NOT scaled EX VIVO and IN VIVO?  


threshold=0.95
dict_false_positives={}
dict_true_positives={}
dict_false_negatives={}
dict_true_negatives={}

for protein in proteins:
    dict_false_positives[protein]= 0
    dict_true_positives[protein]= 0
    dict_false_negatives[protein]= 0
    dict_true_negatives[protein]= 0

del protein


for protein in proteins:

    #for index, row in CELF1.iterrows():
    protein_actual_pos=alldata[alldata[protein] == 1]
    protein_actual_neg=alldata[alldata[protein] == 0]
    

    
    
    for index, row in protein_actual_pos.iterrows():

        #TP is a position that has binding site for a given protein  as indicated by CLIPD, 
        #and for which diffBUM-HMM predicts a differential reactive nucleotide either ex vivo or in vivo
        if row['ex_vivo'] >= threshold or row['in_vivo'] >= threshold:
            dict_true_positives[protein]= dict_true_positives[protein]+1

        #FN is a position that has binding site for a given protein as indicated by CLIPD,
        #and for which diffBUM-HMM does NOT predict a differential reactive nucleotide either ex vivo or in vivo
        else:
            dict_false_negatives[protein]= dict_false_negatives[protein]+1

    for index, row in protein_actual_neg.iterrows():
        
      
    
        if row['ex_vivo'] >= threshold or row['in_vivo'] >= threshold:
             #FP is a position that has NO binding site for a given protein as indicated by CLIPD,
     #and for which diffBUM-HMM predicts a differential reactive nucleotide either ex vivo or in vivo 
            dict_false_positives[protein]= dict_false_positives[protein]+1
        
        else:
            #TN is a position that has NO binding site for a given protein as indicated by CLIPD,
     #and for which diffBUM-HMM does NOT predicts a differential reactive nucleotide either ex vivo or in vivo 
            dict_true_negatives[protein]= dict_true_negatives[protein]+1

print('TP' +str(dict_true_positives))
print('FN' +str(dict_false_negatives))
print('FP'+ str(dict_false_positives))
print('TN' + str(dict_true_negatives))

del protein

#TPR = TP/(TP+FN)
#FPR = FP/(FP+TN)

dict_false_positive_rate={}
dict_true_positive_rate={}

for protein in proteins:
    dict_false_positive_rate[protein]= 0
    dict_true_positive_rate[protein]= 0

del protein

for protein in proteins:
    
    dict_true_positive_rate[protein]=dict_true_positives[protein]/(dict_true_positives[protein]+dict_false_negatives[protein])
    dict_false_positive_rate[protein]= dict_false_positives[protein]/(dict_false_positives[protein]+dict_true_negatives[protein])


print('TPR ' + str(dict_true_positive_rate))
print('FPR ' + str(dict_false_positive_rate))
   


TP{'CELF1': 47, 'FUS': 306, 'HuR': 76, 'PTBP1': 35, 'RBFOX2': 3, 'TARDBP': 48}
FN{'CELF1': 755, 'FUS': 3388, 'HuR': 1073, 'PTBP1': 569, 'RBFOX2': 63, 'TARDBP': 486}
FP{'CELF1': 1264, 'FUS': 1005, 'HuR': 1235, 'PTBP1': 1276, 'RBFOX2': 1308, 'TARDBP': 1263}
TN{'CELF1': 15852, 'FUS': 13219, 'HuR': 15534, 'PTBP1': 16038, 'RBFOX2': 16544, 'TARDBP': 16121}
TPR {'CELF1': 0.05860349127182045, 'FUS': 0.0828370330265295, 'HuR': 0.06614447345517842, 'PTBP1': 0.057947019867549666, 'RBFOX2': 0.045454545454545456, 'TARDBP': 0.0898876404494382}
FPR {'CELF1': 0.07384903014723067, 'FUS': 0.07065523059617548, 'HuR': 0.07364780249269486, 'PTBP1': 0.07369758576874205, 'RBFOX2': 0.07326910150123235, 'TARDBP': 0.07265301426599172}
